In [48]:
import os,shutil
files = folders = 0
for _, dirnames, filenames in os.walk("../anomaly_karyotype/"):
  # ^ this idiom means "we won't be using this value"
    files += len(filenames)
    folders += len(dirnames)
print ("{:,} files, {:,} folders".format(files, folders))
#around 180 abnormal folders, use 20 normal folders

371 files, 1 folders


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [11]:
from keras.datasets import mnist
dirpath="../anomaly_karyotype/translocation_gan/"
x=[]
img = data.imread(dirpath+"1.png")
print(img.shape)
# print(type(X_train[0][0]))

(128, 128, 3)


In [ ]:
shutil.make_archive("../rawdata.zip", 'zip', "../rawdata")

In [2]:
import zipfile
with zipfile.ZipFile("../anomaly_karyotype.zip", 'r') as zip_ref:
    zip_ref.extractall("../anomaly_karyotype")

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape,Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.recurrent import LSTM,GRU
from keras.preprocessing import image
from keras.models import Model
from keras.optimizers import SGD,Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
from keras import backend as K
import os,random,shutil
from skimage import io,color,data,measure,morphology
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import numpy as np
# K.set_image_dim_ordering('tf')

In [2]:
import h5py
from sklearn.model_selection import train_test_split
image_path = "../anomaly_karyotype/translocation_single/"     # images
normal_path="../anomaly_karyotype/normal_single/"
y=[]
x = []
count=0
for _, dirnames, filenames in os.walk("../anomaly_karyotype/translocation_single",topdown=False):
    for d in dirnames:
        folder = image_path + d+'/'
        for _, dirnames2, filenames2 in os.walk(folder,topdown=False):
            for d in dirnames2:
                subfolder=folder+d+'/'
                if os.path.isdir(subfolder):
                    images = [f for f in os.listdir(subfolder) if os.path.isfile(os.path.join(subfolder, f))]
                    for j in images:
                        img = data.imread(os.path.join(subfolder,j),as_gray=True)
                        try:
                            img = resize(img,(128,128,3))
                        except BaseException:
                            print(os.path.join(subfolder, j))
                        if j[-5]=='t':
                            img2=np.fliplr(img)
                            y.append(1)
                            y.append(1)
                            x.append(img)
                            x.append(img2)
#                             io.imsave("../anomaly_karyotype/translocation_gan/"+str(count)+".png",img)
#                             count+=1
random.shuffle(x)
xtest=x[-74:]
ytest=y[-74:]
x=x[:-74]
y=y[:-74]
print(len(x))
# dirpath="../anomaly_karyotype/gan_filtered"
# for _, dirnames, filenames in os.walk("../anomaly_karyotype/gan_filtered",topdown=False):
#             images = [os.path.join(dirpath, f) for f in filenames if os.path.isfile(os.path.join(dirpath, f))]
#             for j in images:
#                 img = data.imread(j,as_gray=True)
#                 img = resize(img,(128,128,3))
#                 x.append(img)
#                 y.append(1)
normal=[]#35 normal folders, 16 for training, 19 for test
for _, dirnames, filenames in os.walk("../anomaly_karyotype/normal_single",topdown=False):
    for d in dirnames:
        folder = normal_path + d+'/'
        if os.path.isdir(folder):
            images = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
            for j in images:
                img = data.imread(os.path.join(folder,j),as_gray=True)
                try:
                    img = resize(img,(128,128,3))
                except BaseException:
                    print(os.path.join(folder, j))
                normal.append(img)
                img2=np.fliplr(img)
                normal.append(img2)
print(len(y),len(ytest),len(normal))

660
660 74 1540


In [49]:
xtest_normal=normal[1140:]
ytest_normal=[]
x=x+normal[:1140]
for i in range(1140):
    y.append(0)
for i in range(400):
    ytest_normal.append(0)

In [3]:
xtest_normal=normal[660:]
ytest_normal=[]
x=x+normal[:660]
for i in range(660):
    y.append(0)
for i in range(880):
    ytest_normal.append(0)

In [4]:
x_train = np.array(x)
y_train=np.array(y)
x_test=np.array(xtest)
y_test=np.array(ytest)
x_test_normal=np.array(xtest_normal)
y_test_normal=np.array(ytest_normal)
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]
# indices = np.arange(x_test.shape[0])
# np.random.shuffle(indices)
# x_test = x_test[indices]
# y_test = y_test[indices]
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_test_normal= x_test_normal.astype('float32')
print(x_train.shape,y_train.shape,x_test.shape,x_test_normal.shape)

(1320, 128, 128, 3) (1320,) (74, 128, 128, 3) (880, 128, 128, 3)


In [5]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model
base_model=VGG19(weights= None, include_top=False, input_shape= (128,128,3))
x = base_model.output
x=Flatten()(x)
predictions = Dense(2, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
adam=Adam(lr=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# checkpoint = ModelCheckpoint('dense121_0801.h5', monitor='val_loss',verbose=1,save_best_only=True,mode='min')
# history=model.fit(x_train, y_train,batch_size=32,epochs=10,validation_split=0.1,callbacks = [checkpoint])
history=model.fit(x_train, y_train,batch_size=32,epochs=10,validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 1188 samples, validate on 132 samples
Epoch 1/10


In [ ]:
history=model.fit(x_train, y_train,batch_size=32,epochs=5,validation_split=0.1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
# model.load_weights("dense201.h5")
y_pred = model.predict(x_test,batch_size=16, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_bool))

In [ ]:
y_pred = model.predict(x_test_normal,batch_size=16, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(y_test_normal, y_pred_bool))